In [ ]:
from numpy import exp
from numpy import sqrt
from numpy import cos
from numpy import e
from numpy import pi
from typing import List
import random

In [ ]:
A = 20
B = 0.2
C = 2 * pi
D = 30

In [ ]:
def ackley_funct(x: List[float]):
  return - A * exp(-B * sqrt(sum(map(lambda n: n ** 2, x)) / D)) \
          - exp(sum(map(lambda n: cos(C * n), x)) / D) \
          + A + exp(1)

In [ ]:
print(f'{ackley_funct([0] * 30) : .15f}') 

 0.000000000000000


In [ ]:
def generate_individual():
  x = [0] * D

  for i in range(D):
    x[i] = random.uniform(-15, 15)